<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP 2.0
  </div> 


<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
    <font color=orange>I - 1</font> Tokenization
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 


  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

<a id="plan"></a>

***
<div style="font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Table of Content
  </div> 

$\qquad \qquad \bullet$ [Regex](#regex) <br>
$\qquad \qquad \bullet$ [NLTK](#nltk) <br>
$\qquad \qquad \bullet$ [Spacy](#spacy) <br>
$\qquad \qquad \bullet$ [WordPiece](#word_piece) <br>
$\qquad \qquad \bullet$ [Byte Pair Encoding](#bpe) <br>



In [1]:
import os
import unicodedata
import torch
import inspect
import multiprocessing

# data
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling

# tokenizers
from tokenizers.pre_tokenizers import Whitespace
# base tokenizers
from tokenizers import Tokenizer
from tokenizers.models import (
    BPE,
    Unigram,
    WordLevel,
    WordPiece,
) 
# base tokenizer trainers
from tokenizers.trainers import (
    BpeTrainer,
    UnigramTrainer,
    WordLevelTrainer,
    WordPieceTrainer,
)
# other special tokenizers, see the list at https://github.com/huggingface/tokenizers/tree/master/bindings/python/py_src/tokenizers/implementations
from tokenizers.implementations import (
    CharBPETokenizer,          # The original BPE
    ByteLevelBPETokenizer,     # The byte level version of the BPE
    SentencePieceBPETokenizer, # A BPE implementation compatible with the one used by SentencePiece
    BertWordPieceTokenizer,    # The famous Bert tokenizer, using WordPiece
) 
# model-specific tokenizers
from transformers import GPT2TokenizerFast, RobertaTokenizerFast, DebertaV2Tokenizer

# token embedding
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile

# models
from transformers import RobertaConfig, RobertaModel, RobertaForMaskedLM, pipeline

# model trainer
from transformers import Trainer, TrainingArguments

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Packages

In [2]:
from __future__ import unicode_literals, print_function, division
import sys
import warnings
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode
import itertools
import matplotlib
import matplotlib.pyplot as plt


# for special math operation
from sklearn.preprocessing import normalize


# for manipulating data 
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import bcolz # see https://bcolz.readthedocs.io/en/latest/intro.html
import pickle


# for text processing
import gensim
from gensim.models import KeyedVectors
#import spacy
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer


# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

warnings.filterwarnings("ignore")
print('python version :', sys.version)
print('pytorch version :', torch.__version__)
print('DL device :', device)

python version : 3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
pytorch version : 1.5.0
DL device : cuda


In [5]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_data = os.path.join(path_to_repo, 'data', 'news_auto')
path_to_save = os.path.join(path_to_repo, 'saves', 'news_auto')
#path_to_save = os.path.join(path_to_repo, 'saves', '20_news_group')

path_to_roberta = os.path.join(path_to_save, 'roberta')

<a id="corpus"></a>

# 1 Corpus

[Back to top](#plan)

### $\bullet$ News Auto

In [6]:
df_newsauto = pd.read_excel(os.path.join(path_to_data, "news_auto_texts.xlsx"), engine = 'openpyxl')
df_newsauto = df_newsauto.fillna('')

In [7]:
urls_root = list(set(['/'.join(url.split('/')[:3]) for url in df_newsauto.url.tolist()]))

urls_trn, urls_tst = train_test_split(urls_root, test_size = 0.1, random_state = 4242)
print(len(urls_trn), len(urls_tst))

529 59


In [8]:
filter_trn = df_newsauto.url.apply(lambda url: '/'.join(url.split('/')[:3]) in urls_trn)
filter_tst = df_newsauto.url.apply(lambda url: '/'.join(url.split('/')[:3]) in urls_tst)

texts_trn = [t for t in df_newsauto[filter_trn].text.tolist() if t]
texts_tst = [t for t in df_newsauto[filter_tst].text.tolist() if t]
print(len(texts_trn), len(texts_tst))

texts_trn = unicodedata.normalize("NFKD", '\n\n'.join(texts_trn))
texts_tst = unicodedata.normalize("NFKD", '\n\n'.join(texts_tst))

3590 275


In [9]:
with open(os.path.join(path_to_data, "news_auto_texts_trn.txt"), "w", encoding='utf-8') as text_file:
    text_file.write(texts_trn)

with open(os.path.join(path_to_data, "news_auto_texts_tst.txt"), "w", encoding='utf-8') as text_file:
    text_file.write(texts_tst)

In [10]:
# re-import texts
with open(os.path.join(path_to_data, "news_auto_texts_trn.txt"), "r", encoding='utf-8') as text_file:
    texts_trn = text_file.read()
    texts_trn = texts_trn.split('\n\n')

with open(os.path.join(path_to_data, "news_auto_texts_tst.txt"), "r", encoding='utf-8') as text_file:
    texts_tst = text_file.read()
    texts_tst = texts_tst.split('\n\n')

In [11]:
len(texts_trn), len(texts_tst)

(3590, 275)

### $\bullet$ 20 news group

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
newsgroups_trn = fetch_20newsgroups(subset = 'train', remove = ('headers', 'footers', 'quotes'))
newsgroups_tst = fetch_20newsgroups(subset = 'test',  remove = ('headers', 'footers', 'quotes'))

In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) # 
    return s

def cleanNum(s) :
    s = re.sub('[\.!?]+ ', ' . ', s)
    s = re.sub(',+ ', ' , ', s)
    s = re.sub(' [0-9]*\.[0-9] ', ' FLOAT ', ' ' + s + ' ').strip()
    s = re.sub(' [0-9,]*[0-9] ', ' INT ', ' ' + s + ' ').strip()
    return s

def trueWord(w) :
    return len(w)>0 and re.sub('[^a-zA-Z0-9.,]', '', w) != ''

def prepareCorpus(corpus) :
    corpus = [normalizeString(s) for s in corpus]
    corpus = [cleanNum(s) for s in corpus]
    corpus = [nltk.tokenize.word_tokenize(s) for s in corpus]
    corpus = [[w for w in s if trueWord(w)] for s in corpus]
    corpus = [s for s in corpus if len(s) < 1000] # a lot of crapy words are accumulated among few texts
    return corpus

In [ ]:
corpus = prepareCorpus(newsgroups_trn.data)
len(corpus)

<a id="tokenizers"></a>

# 2 Tokenizers

[Back to top](#plan)

### $\bullet$ Pretrained tokenizers

In [22]:
# # use pre-fitted tokenizer for DeBERTa (BERT-like adaptation of GPT2 tokenizer, a particular BPE tokenizer)
# tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v2-xlarge")

### $\bullet$ GPT-2 tokenizer

Tokenizer based on a byte-level BPE tokenizer. The list of special tokens is determined by the [GPT2TokenizerFast](https://huggingface.co/transformers/_modules/transformers/models/gpt2/tokenization_gpt2_fast.html#GPT2TokenizerFast) class.

In [96]:
path_to_vocab_gpt2 = os.path.join(path_to_save, 'gpt2', 'vocabulary')

# instantiate, fit and export GPT2 tokenizer
tokenizer_gpt2 = ByteLevelBPETokenizer()
tokenizer_gpt2.train(
    files = os.path.join(path_to_data, "news_auto_texts_trn.txt"), 
    vocab_size = 5000,
    special_tokens = ['<|endoftext|>'],
)
tokenizer_gpt2.save_model(path_to_vocab_gpt2)

['C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\news_auto\\gpt2\\vocabulary\\vocab.json',
 'C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\news_auto\\gpt2\\vocabulary\\merges.txt']

In [97]:
#re-import tokenizer into a more suitable class for model training
tokenizer_gpt2 = GPT2TokenizerFast.from_pretrained(path_to_vocab_gpt2, max_len = 512)

In [98]:
# "begin of word" is encoded by the special 'Ġ' character, see
# https://discuss.huggingface.co/t/bpe-tokenizers-and-spaces-before-words/475
txt = "Hello, y'all! How are you ?"
output = tokenizer_gpt2(txt)
tokens = output.tokens

print(txt)
print(tokens)

Hello, y'all! How are you ?
<bound method BatchEncoding.tokens of {'input_ids': [40, 457, 79, 12, 569, 7, 499, 1, 2604, 419, 817, 221, 31], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}>


### $\bullet$ Roberta tokenizer

Roberta tokenizer is a subclass of GPT2 tokenizer. The list of special tokens is determined by the [RobertaTokenizerFast](https://huggingface.co/transformers/_modules/transformers/models/roberta/tokenization_roberta_fast.html#RobertaTokenizerFast) class.

In [12]:
path_to_vocab_roberta = os.path.join(path_to_roberta, 'vocabulary')

In [20]:
# instantiate, fit and export Roberta tokenizer
vocab_size = 5000
special_tokens = ["<s>", "<pad>", "</s>", "<unk>", "<mask>"]

tokenizer_roberta = ByteLevelBPETokenizer()
tokenizer_roberta.train(
    files = os.path.join(path_to_data, "news_auto_texts_trn.txt"), 
    vocab_size = vocab_size,
    special_tokens = special_tokens,
)
tokenizer_roberta.save_model(path_to_vocab_roberta)

['C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\data\\vocab\\news_auto_roberta\\vocab.json',
 'C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\data\\vocab\\news_auto_roberta\\merges.txt']

In [13]:
#re-import tokenizer into a more suitable class for model training
tokenizer_roberta = RobertaTokenizerFast.from_pretrained(path_to_vocab_roberta, max_len = 512)

In [89]:
tokenizer_roberta.save_pretrained(os.path.join(path_to_roberta, "tokenizer"))

('C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\Tokenizer\\news_auto_roberta\\tokenizer_config.json',
 'C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\Tokenizer\\news_auto_roberta\\special_tokens_map.json',
 'C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\Tokenizer\\news_auto_roberta\\vocab.json',
 'C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\Tokenizer\\news_auto_roberta\\merges.txt',
 'C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\Tokenizer\\news_auto_roberta\\added_tokens.json')

In [14]:
txt = "Hello, y'all! How are you ?"
output = tokenizer_roberta(txt)
tokens = output.tokens

print(txt)
print(tokens)

Hello, y'all! How are you ?
<bound method BatchEncoding.tokens of {'input_ids': [0, 44, 461, 83, 16, 573, 11, 503, 5, 2608, 423, 821, 225, 35, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}>


### $\bullet$ Deberta-v2 tokenizer

Tokenizer based on _SentencePiece_. The list of special tokens is determined by the [DebertaV2Tokenizer](https://huggingface.co/transformers/_modules/transformers/models/deberta_v2/tokenization_deberta_v2.html#DebertaV2Tokenizer) class.

In [99]:
path_to_vocab_deberta_v2 = os.path.join(path_to_save, 'deberta_v2', 'vocabulary')

# instantiate, fit and export Deberta-v2 tokenizer
vocab_size = 5000
special_tokens = ["[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]"]

tokenizer_debertav2 = SentencePieceBPETokenizer()
tokenizer_debertav2.train(
    files = os.path.join(path_to_data, "news_auto_texts_trn.txt"), 
    vocab_size = vocab_size,
    special_tokens = special_tokens,
)
tokenizer_debertav2.save_model(path_to_vocab_deberta_v2)

['C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\news_auto\\deberta_v2\\vocabulary\\vocab.json',
 'C:\\Users\\Jb\\Desktop\\NLP\\BERTology\\saves\\news_auto\\deberta_v2\\vocabulary\\merges.txt']

In [14]:
# #re-import tokenizer into a more suitable class for model training
# not working_
# tokenizer_debertav2 = DebertaV2Tokenizer.from_pretrained(path_to_vocab_deberta_v2, max_len = 512)
# not working either
# tokenizer_debertav2 = DebertaV2Tokenizer(vocab_file = os.path.join(path_to_vocab_deberta_v2, 'vocab.json'))

### $\bullet$ Custom tokenizers

In [288]:
def fit_tokenizer(Model, Trainer, vocab_size, special_tokens):
    model = (
        Model(unk_token = "[UNK]")
        if 'unk_token' in inspect.signature(Model).parameters # for Unigram model
        else Model()
    )
    # instantiate tokenizer, and fit using external trainer
    tokenizer = Tokenizer(model)
    tokenizer.pre_tokenizer = Whitespace()
    trainer = Trainer(vocab_size = vocab_size, special_tokens = special_tokens)
    tokenizer.train(
        files = [os.path.join(path_to_data, "news_auto_texts_trn.txt")], 
        trainer = trainer,
    )
    return tokenizer

In [294]:
vocab_size = 5000
special_tokens = ["[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]"]

bpe_tokenizer = fit_tokenizer(BPE, BpeTrainer, vocab_size, special_tokens)
wdl_tokenizer = fit_tokenizer(WordLevel, WordLevelTrainer, vocab_size, special_tokens)
wdp_tokenizer = fit_tokenizer(WordPiece, WordPieceTrainer, vocab_size, special_tokens)
uni_tokenizer = fit_tokenizer(Unigram, UnigramTrainer, vocab_size, special_tokens)

In [295]:
path_to_bpe = os.path.join(path_to_data, 'vocab', 'news_auto_bpe.json')

# save
bpe_tokenizer.save(path_to_bpe, pretty = True)

# load as base Tokenizer
bpe_tokenizer = Tokenizer.from_file(path_to_bpe)

# load as tokenizer suitable for model training
tokenizer_bpe = PreTrainedTokenizerFast(tokenizer_file = path_to_bpe)

In [299]:
txt = "Hello, y'all! How are you 😁? wanna work at Toyota?"
output = tokenizer_bpe(txt)
tokens = output.tokens

print(txt)
print(tokens)

Hello, y'all! How are you 😁? wanna work at Toyota?
<bound method BatchEncoding.tokens of {'input_ids': [44, 1034, 1067, 16, 93, 11, 1121, 5, 2512, 1092, 1458, 0, 35, 91, 1287, 69, 1223, 1013, 4194, 35], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}>
